In [1]:
# Install required libraries
!pip install transformers datasets torch scikit-learn

In [2]:
# Core libraries
import torch # Torch: tensors, DataLoader, optimizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt #matplotlib: plotting

# Hugging Face
from datasets import load_dataset
from transformers import BertTokenizer, BertForSequenceClassification # Transformers: BERT model and tokenizer
from transformers import get_scheduler

# PyTorch
from torch.optim import AdamW
from torch.utils.data import DataLoader

# Evaluation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report # Scikit-learn: baseline model and evaluation metrics

# Baseline model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Progress bar
from tqdm.auto import tqdm


In [3]:
# Load IMDb dataset: 50k movie reviews, binary sentiment labels (0=negative, 1=positive)
# Chosen because it's a standard benchmark for sentiment analysis
dataset = load_dataset("imdb")

# Display dataset structure
dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [4]:
# Inspect a sample
dataset["train"][0]


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [5]:
# Dataset samples
print("Training samples:", len(dataset["train"]))
print("Test samples:", len(dataset["test"]))


Training samples: 25000
Test samples: 25000


In [6]:
# Extract text and labels
X_train = dataset["train"]["text"]
y_train = dataset["train"]["label"]

X_test = dataset["test"]["text"]
y_test = dataset["test"]["label"]

# Converting text to TF-IDF features; limiting to top 5000 terms to reduce dimensionality
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Logistic Regression is a simple baseline to compare against BERT
baseline_model = LogisticRegression(max_iter=1000)
baseline_model.fit(X_train_tfidf, y_train)

# Fit baseline model on training set, predict on test set
baseline_preds = baseline_model.predict(X_test_tfidf)

# Evaluation
print("Baseline Model Results:")
print(classification_report(y_test, baseline_preds))


Baseline Model Results:
              precision    recall  f1-score   support

           0       0.88      0.88      0.88     12500
           1       0.88      0.88      0.88     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



In [7]:
# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization function
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=256
    )

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove raw text columns and set PyTorch format
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
# Create DataLoaders for training and testing
train_loader = DataLoader(tokenized_datasets["train"], batch_size=16, shuffle=True)
test_loader = DataLoader(tokenized_datasets["test"], batch_size=16)


In [9]:
# Load BERT for sequence classification
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)

# Move model to GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("GPU")
model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [10]:
# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Number of training steps
num_epochs = 2
num_training_steps = num_epochs * len(train_loader)

# Learning rate scheduler
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)


In [11]:
# Training loop
model.train()

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")

    for batch in train_loader:
        # Move batch to device
        batch = {k: v.to(device) for k, v in batch.items()}

        # Rename 'label' to 'labels' (this fixes the error)
        batch["labels"] = batch.pop("label")

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass
        loss.backward()

        # Optimizer step
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)


  0%|          | 0/3126 [00:00<?, ?it/s]


Epoch 1/2

Epoch 2/2


In [ ]:
# Evaluation
model.eval()

predictions = []
true_labels = []

with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        # Rename label key
        labels = batch.pop("label")

        outputs = model(**batch)
        logits = outputs.logits

        preds = torch.argmax(logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())


In [ ]:
# Compute metrics
accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average="weighted")

print("BERT Model Results:")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-score:  {f1:.4f}")


In [ ]:
# Detailed report
print(classification_report(true_labels, predictions))


In [ ]:
results = pd.DataFrame({
    "Model": ["Logistic Regression (TF-IDF)", "BERT Fine-Tuned"],
    "Accuracy": [
        accuracy_score(y_test, baseline_preds),
        accuracy
    ],
    "F1-score": [
        precision_recall_fscore_support(y_test, baseline_preds, average="weighted")[2],
        f1
    ]
})

results
